In [3]:
# -*- coding: utf-8 -*-
"""
도로 중심 위경도 + 반경 내 카테고리 '개수만' 집계하여 CSV 저장
- road_coords.csv : 도로명, 위도, 경도 (좌표 캐시)
- road_category_counts_radius{r}_m.csv : 도로별 카테고리 개수 요약
"""
import os
import time
import json
import math
import pandas as pd
import requests
from typing import Dict, List, Optional

# ================= 설정 =================
API_KEY = os.getenv("KAKAO_API_KEY", "41d56449d3dce5dde7d48ffb261f0e8a")
CITY_PREFIX = "수원시"
TARGET_ROADS = ["팔달로", "권선로", "경수대로", "효원로", "수원천로", "장다리로", "권광로", "금곡로", "정조로", "영통로"]

# 수집 카테고리 (요구사항 고정)
CATEGORIES = {
    "FD6": "음식점",
    "CE7": "카페",
    "BK9": "은행",
    "HP8": "병원",
    "PO3": "관공서",
    "PK6": "주차장",
}

# 검색 반경 (미터)
RADIUS_M = 500

# 수원 중심(바이어스용)
CENTER_X = "127.0286"
CENTER_Y = "37.2636"

# API 요청 간격(초)
SLEEP_SEC = 0.08

HEADERS = {"Authorization": f"KakaoAK {API_KEY}"}

# ================= 유틸 함수 =================
def _request_json(url: str, params: dict, retry: int = 3, backoff: float = 0.6) -> dict:
    """간단 재시도 지원 GET JSON"""
    last_err = None
    for i in range(retry):
        try:
            res = requests.get(url, headers=HEADERS, params=params, timeout=6)
            if res.status_code == 200:
                return res.json()
            last_err = f"status={res.status_code}, body={res.text[:200]}"
        except Exception as e:
            last_err = str(e)
        time.sleep(backoff * (2 ** i))
    raise RuntimeError(f"GET 실패: {url} params={params} err={last_err}")

def get_road_coord(road_name: str) -> Optional[Dict]:
    """도로 중심 좌표 1건만 조회 (키워드 검색)"""
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    params = {
        "query": f"{CITY_PREFIX} {road_name}",
        "x": CENTER_X, "y": CENTER_Y, "radius": 20000,
        "size": 1, "page": 1,
    }
    try:
        data = _request_json(url, params)
        docs = data.get("documents", [])
        if docs:
            d = docs[0]
            return {
                "road": road_name,
                "lat": float(d["y"]),
                "lng": float(d["x"]),
                "query_addr": d.get("address_name", ""),
            }
    except Exception as e:
        print(f"❌ 좌표 실패: {road_name} -> {e}")
    return None

def count_nearby(lat: float, lng: float, category_group_code: str, radius_m: int = 500) -> int:
    """카테고리 API meta.total_count만 사용해서 개수 집계"""
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    params = {
        "category_group_code": category_group_code,
        "x": lng, "y": lat,
        "radius": radius_m,
        "size": 1, "page": 1,  # 목록 불필요, meta.total_count만 확인
    }
    try:
        data = _request_json(url, params)
        meta = data.get("meta", {})
        # 카카오가 total_count를 45페이지*15 = 675로 캡할 수 있음(검색 한계)
        # 하지만 반경 500m 내에서는 일반적으로 충분. 필요시 반경 줄이거나 그리드 샘플링.
        return int(meta.get("total_count", 0))
    except Exception as e:
        print(f"❌ 개수 조회 실패: code={category_group_code} ({lat},{lng}) -> {e}")
        return 0

# ================= 메인 로직 =================
def load_cached_coords(path="road_coords.csv") -> Dict[str, Dict]:
    """이미 저장된 좌표 캐시 읽기 (있으면 재사용)"""
    if not os.path.exists(path):
        return {}
    try:
        df = pd.read_csv(path)
        out = {}
        for _, r in df.iterrows():
            out[str(r["road"])] = {
                "road": r["road"],
                "lat": float(r["lat"]),
                "lng": float(r["lng"]),
                "query_addr": r.get("query_addr", ""),
            }
        return out
    except Exception:
        return {}

def save_coords_cache(coords: List[Dict], path="road_coords.csv"):
    df = pd.DataFrame(coords)
    df.to_csv(path, index=False, encoding="utf-8-sig")
    print(f"💾 좌표 캐시 저장: {path} (rows={len(df)})")

def collect_counts(radius_m: int = RADIUS_M):
    # 1) 좌표 캐시 활용 + 부족분 조회
    cache = load_cached_coords()
    coords: List[Dict] = []
    for road in TARGET_ROADS:
        if road in cache:
            coords.append(cache[road])
        else:
            c = get_road_coord(road)
            if c:
                coords.append(c)
            else:
                print(f"⚠️ 좌표 없음: {road}")
            time.sleep(SLEEP_SEC)
    # 캐시 갱신
    save_coords_cache(coords)

    # 2) 카테고리별 개수 집계(업체명 X)
    rows = []
    for rc in coords:
        row = {"road": rc["road"], "lat": rc["lat"], "lng": rc["lng"]}
        for code, name in CATEGORIES.items():
            cnt = count_nearby(rc["lat"], rc["lng"], code, radius_m)
            row[name] = cnt
            time.sleep(SLEEP_SEC)
        row["음식+카페"] = row["음식점"] + row["카페"]
        row["총합"] = sum(row[name] for name in CATEGORIES.values())
        rows.append(row)

    df_counts = pd.DataFrame(rows).sort_values("총합", ascending=False).reset_index(drop=True)
    out_path = f"road_category_counts_radius{radius_m}_m.csv"
    df_counts.to_csv(out_path, index=False, encoding="utf-8-sig")

    print("\n📦 저장 완료")
    print(f"  - road_coords.csv")
    print(f"  - {out_path}")
    print("\n▶ 상위 5개 미리보기:")
    print(df_counts.head())

    return df_counts

if __name__ == "__main__":
    if not API_KEY or API_KEY.startswith("여기에_"):
        raise SystemExit("❗ KAKAO_API_KEY를 설정하세요. (환경변수 KAKAO_API_KEY 또는 스크립트 상단 API_KEY)")
    _ = collect_counts(RADIUS_M)


💾 좌표 캐시 저장: road_coords.csv (rows=10)

📦 저장 완료
  - road_coords.csv
  - road_category_counts_radius500_m.csv

▶ 상위 5개 미리보기:
   road        lat         lng  음식점   카페  은행   병원  관공서  주차장  음식+카페    총합
0   효원로  37.262885  127.030425  714  149  56  129    2   71    863  1121
1   권광로  37.263906  127.032310  670  142  57  126    2   70    812  1067
2  수원천로  37.287506  127.018074  227  128   8    5    1   17    355   386
3   금곡로  37.274591  126.956151  252   51  11   36    3   19    303   372
4   영통로  37.238421  127.057025  191   39  13   45    4   11    230   303


In [5]:
import pandas as pd

# 1) 도로 좌표 JSON 변환
coords_csv = "road_coords.csv"
coords_json = "road_coords.json"

df_coords = pd.read_csv(coords_csv)
df_coords.to_json(coords_json, orient="records", force_ascii=False, indent=2)
print(f"✅ {coords_json} 저장 완료")

# 2) 도로별 카테고리 개수 JSON 변환
counts_csv = "road_category_counts_radius500_m.csv"
counts_json = "road_category_counts_radius500_m.json"

df_counts = pd.read_csv(counts_csv)
df_counts.to_json(counts_json, orient="records", force_ascii=False, indent=2)
print(f"✅ {counts_json} 저장 완료")

✅ road_coords.json 저장 완료
✅ road_category_counts_radius500_m.json 저장 완료


### 데이터가 .CSV가 아니라 .JSON형태여야 해서 변환

In [9]:
#단속데이터 결측치 제거하고 json파일로 저장
# -*- coding: utf-8 -*-
"""
주정차 단속 데이터 정리 → JSON (카카오맵 렌더용)
- 입력: CSV (필드: 집계년도, 시군명, 관리기관명, 단속일시정보, 단속방법, 데이터기준일자, 단속장소, lat, lon)
- 출력: data/violations.json  (records: [{lat, lon, type, hour, count}, ...])
"""
import os
import pandas as pd
import numpy as np

# ===== 경로/파일명 =====
INPUT_CSV = "주정차위반단속_위경도.csv" 
OUTPUT_JSON = os.path.join("data", "violations.json")
os.makedirs("data", exist_ok=True)

# ===== 1) 로드 + 필요한 컬럼만 =====
# 인코딩 자동 시도
for enc in ("utf-8", "cp949", "euc-kr"):
    try:
        df = pd.read_csv(INPUT_CSV, encoding=enc)
        break
    except Exception:
        df = None
if df is None:
    raise RuntimeError("CSV 파일을 읽지 못했습니다. 인코딩/경로를 확인하세요.")

need_cols = ['집계년도','시군명','관리기관명','단속일시정보','단속방법','단속장소','lat','lon']
missing = [c for c in need_cols if c not in df.columns]
if missing:
    raise ValueError(f"필수 컬럼 누락: {missing}")

df = df[need_cols].copy()

# ===== 2) 기본 전처리 =====
# 날짜 → datetime
df['단속일시정보'] = pd.to_datetime(df['단속일시정보'], errors='coerce')

# lat/lon 숫자화
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

# 결측 제거: (시간, 단속방법, lat, lon) 모두 있어야 함
before = len(df)
df = df.dropna(subset=['단속일시정보', '단속방법', 'lat', 'lon'])
after = len(df)
print(f"결측 제거: {before - after}건 제거, 남은 {after}건")

# (선택) 수원 근처 좌표만 유지 (이상치 제거용: 대략 bounding box)
# 수원 대략: lat 37.18~37.36, lon 126.92~127.12
df = df[(df['lat'].between(37.18, 37.36)) & (df['lon'].between(126.92, 127.12))]

# ===== 3) 파생 컬럼: hour, type =====
df['hour'] = df['단속일시정보'].dt.hour.astype(int)

# 단속방법 정규화(공백/대소문자 등)
def norm_type(x: str) -> str:
    if not isinstance(x, str):
        return "기타"
    y = x.strip()
    # 필요시 치환 룰을 여기 추가
    # 예) '주민신고' → '주민신고제'
    if y in ("주민신고", "주민 신고"):
        y = "주민신고"
    return y

df['type'] = df['단속방법'].astype(str).map(norm_type)

# ===== 4) 좌표 집계(클러스터링은 안 하고 '정확 좌표' 기준으로 집계) =====
#   render.js는 (lat, lon, hour, type) 단위로 count 사용
group_cols = ['lat', 'lon', 'hour', 'type']
agg = (
    df.groupby(group_cols, dropna=False)
      .size()
      .reset_index(name='count')
      .sort_values('count', ascending=False)
      .reset_index(drop=True)
)

# ===== 5) JSON 저장 =====
# orient="records" → [{lat:.., lon:.., hour:.., type:.., count:..}, ...]
agg.to_json(OUTPUT_JSON, orient="records", force_ascii=False)
print(f"✅ 저장 완료: {OUTPUT_JSON} (rows={len(agg)})")

# ===== 6) 참고: 필터 옵션이 필요하면 한 번에 함께 출력(선택) =====
summary = (
    df.groupby('type')
      .size()
      .reset_index(name='n')
      .sort_values('n', ascending=False)
)
print("\n단속방법 분포(상위 10):")
print(summary.head(10).to_string(index=False))

hours_cnt = df['hour'].value_counts().sort_index()
print("\n시간대 분포(0~23):")
print(hours_cnt.to_string())


결측 제거: 44329건 제거, 남은 325407건
✅ 저장 완료: data\violations.json (rows=46797)

단속방법 분포(상위 10):
 type      n
  고정형 119853
  주행형 102803
국민신문고  91344
   보행   9291
주민신고제   2116

시간대 분포(0~23):
hour
0      1846
1      1058
2       783
3       526
4       563
5      1672
6      2257
7      8773
8     20828
9     25836
10    34583
11    10428
12     7849
13     8770
14    46115
15    33535
16    22955
17    19617
18    14936
19    24864
20    19255
21     9475
22     5312
23     3571
